In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np
import pandas as pd
import dask.dataframe as dd

import plotly.express as px
import plotly.graph_objects as go

import re


In [2]:
df = dd.read_csv('Iowa_Liquor_Sales.csv',assume_missing=True,engine='python',usecols=[
    'Invoice/Item Number','Date','Store Number','Address','City','Zip Code','County Number','County',
    'Category','Category Name','Vendor Number','Vendor Name','Item Number','Item Description',
    'Pack', 'Bottle Volume (ml)', 'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold'],
    dtype={'Bottle Volume (ml)': 'float64','Bottles Sold': 'float64','Item Number': 'float64',
    'Pack': 'float64','Store Number': 'float64','Vendor Number': 'object'},
    warn_bad_lines=False,error_bad_lines=False)
# Sale (Dollars) = Retail * Sold
# Volumne Sold (Liters) = Volume (ml) * Sold / 1000
# who uses gallons? 

# df = dd.read_csv('Iowa_Liquor_Sales_50k.csv', assume_missing=True, engine='python')
df.head()


/Users/lyndonf/Desktop/LiquorSales2022/.venv/lib/python3.8/site-packages/dask/dataframe/io/csv.py:613: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  head = reader(BytesIO(b_sample), nrows=sample_rows, **head_kwargs)
/Users/lyndonf/Desktop/LiquorSales2022/.venv/lib/python3.8/site-packages/dask/dataframe/io/csv.py:613: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  head = reader(BytesIO(b_sample), nrows=sample_rows, **head_kwargs)
/Users/lyndonf/Desktop/LiquorSales2022/.venv/lib/python3.8/site-packages/dask/dataframe/io/csv.py:182: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = reader(bio, **kwargs)
/Users/lyndonf/Desktop/LiquorSales2022/.venv/lib/python3.8/site-packages/dask/dataframe/io/csv.py:182: Futur

,Invoice/Item Number,Date,Store Number,Address,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold
0,S29198800001,11/20/2015,2191.0,1013 MAIN,KEOKUK,52632,56.0,Lee,NaN,NaN,255,Wilson Daniels Ltd.,297.0,Templeton Rye w/Flask,6.0,750.0,$18.09,$27.14,6.0
1,S29195400002,11/21/2015,2205.0,900 E WASHINGTON,CLARINDA,51632,73.0,Page,NaN,NaN,255,Wilson Daniels Ltd.,297.0,Templeton Rye w/Flask,6.0,750.0,$18.09,$27.14,12.0
2,S29050300001,11/16/2015,3549.0,1414 48TH ST,FORT MADISON,52627,56.0,Lee,NaN,NaN,130,Disaronno International LLC,249.0,Disaronno Amaretto Cavalli Mignon 3-50ml Pack,20.0,150.0,$6.40,$9.60,2.0
3,S28867700001,11/04/2015,2513.0,812 S 1ST AVE,IOWA CITY,52240,52.0,Johnson,NaN,NaN,65,Jim Beam Brands,237.0,Knob Creek w/ Crystal Decanter,3.0,1750.0,$35.55,$53.34,3.0
4,S29050800001,11/17/2015,3942.0,104 HIGHWAY 30 WEST,TOLEDO,52342,86.0,Tama,NaN,NaN,130,Disaronno International LLC,249.0,Disaronno Amaretto Cavalli Mignon 3-50ml Pack,20.0,150.0,$6.40,$9.60,2.0


In [3]:
df.columns

Index(['Invoice/Item Number', 'Date', 'Store Number', 'Address', 'City',
       'Zip Code', 'County Number', 'County', 'Category', 'Category Name',
       'Vendor Number', 'Vendor Name', 'Item Number', 'Item Description',
       'Pack', 'Bottle Volume (ml)', 'State Bottle Cost',
       'State Bottle Retail', 'Bottles Sold'],
      dtype='object')

In [4]:
df.isna().sum().compute()

/Users/lyndonf/Desktop/LiquorSales2022/.venv/lib/python3.8/site-packages/dask/dataframe/io/csv.py:182: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = reader(bio, **kwargs)
/Users/lyndonf/Desktop/LiquorSales2022/.venv/lib/python3.8/site-packages/dask/dataframe/io/csv.py:182: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = reader(bio, **kwargs)
/Users/lyndonf/Desktop/LiquorSales2022/.venv/lib/python3.8/site-packages/dask/dataframe/io/csv.py:182: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = reader(bio, **kwargs)
/Users/lyndonf/Desktop/LiquorSales2022/.venv/lib/python3.8/site-packages/dask/dataframe/io/csv.py:182: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in

Invoice/Item Number        0
Date                      23
Store Number              23
Address                 2399
City                    2398
Zip Code                2443
County Number          79201
County                 79201
Category                8043
Category Name          16109
Vendor Number             26
Vendor Name               24
Item Number               23
Item Description          23
Pack                      23
Bottle Volume (ml)        23
State Bottle Cost         60
State Bottle Retail       60
Bottles Sold              50
dtype: int64

In [6]:
df[df['Bottles Sold'].isna()].compute()

/Users/lyndonf/Desktop/LiquorSales2022/.venv/lib/python3.8/site-packages/dask/dataframe/io/csv.py:182: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = reader(bio, **kwargs)
/Users/lyndonf/Desktop/LiquorSales2022/.venv/lib/python3.8/site-packages/dask/dataframe/io/csv.py:182: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = reader(bio, **kwargs)
/Users/lyndonf/Desktop/LiquorSales2022/.venv/lib/python3.8/site-packages/dask/dataframe/io/csv.py:182: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = reader(bio, **kwargs)
/Users/lyndonf/Desktop/LiquorSales2022/.venv/lib/python3.8/site-packages/dask/dataframe/io/csv.py:182: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in